In [1]:
import sys
sys.path.append("/home/abhinand/dev/ai/wizardlm_langchain")

from wizardlm_langchain.model import load_quantized_model
from wizardlm_langchain.helpers import AttributeDict, get_available_memory

import accelerate

from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

/home/abhinand/miniconda3/envs/wizardlm_langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not import azure.core python package.


In [2]:
GPTQ_MODEL_DIR = "../models/"
MODEL_NAME = "wizardLM-7B-GPTQ"

In [3]:
args = {
    "wbits": 4,
    "groupsize": 128,
    "model_type": "llama",
    "model_dir": GPTQ_MODEL_DIR,
}

model, tokenizer = load_quantized_model(MODEL_NAME, args=AttributeDict(args))

Path to Model: ../models/wizardLM-7B-GPTQ


/home/abhinand/miniconda3/envs/wizardlm_langchain/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/abhinand/miniconda3/envs/wizardlm_langchain/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/abhinand/miniconda3/envs/wizardlm_langchain/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: Ty

In [8]:
# max_memory = {
#     0: "15360MiB",
#     'cpu': "12GiB"
# }

max_memory = {
    0: 6000000000,
    'cpu': 13000000000
}

In [9]:
device_map = accelerate.infer_auto_device_map(
    model, 
    max_memory=max_memory, 
    no_split_module_classes=["LlamaDecoderLayer"]
)
model = accelerate.dispatch_model(
    model, 
    device_map=device_map, 
    offload_buffers=True
)

In [10]:
model.get_memory_footprint() / (1024 * 1024)

3874.857421875

In [11]:
llm_pipeline = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    device_map=device_map
)

local_llm = HuggingFacePipeline(pipeline=llm_pipeline)

In [12]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt, 
    llm=local_llm
)

In [13]:
print(llm_chain.run('Who invented the light bulb?'))

 The light bulb is a device that produces light by using electricity. It was invented by many people over the years, but the most famous inventor of the light bulb is Thomas Edison. He invented the first practical and commercially successful light bulb in 1879.


In [14]:
print(llm_chain.run('What started the World War 2?'))

 The Treaty of Versailles, which ended World War 1, imposed harsh penalties on Germany, including the payment of reparations and the loss of territory. Germany was in economic crisis and faced political unrest, which led to the rise of Adolf Hitler and the Nazi Party in Germany. Hitler wanted to regain lost territory and restore German pride, which led to his aggressive foreign policy and the expansion of Nazi Germany. This caused tension and conflict with neighboring countries, ultimately leading to the outbreak of World War 2 in 1939.


In [15]:
print(llm_chain.run('When did the Australian bush fires happen?'))

 The Australian bush fires happened in the summer of 2019-2020.


In [16]:
print(llm_chain.run('How much fat does almonds have?'))

 Almonds are a type of nut, which means they are a fruit that contains a single seed. Fruits are typically high in fiber and low in fat. However, almonds are also a good source of healthy fats, specifically monounsaturated and polyunsaturated fats. 

One ounce of almonds (about 28 nuts) contains around 16 grams of fat, with around 13 grams of that being healthy monounsaturated fats. So, while almonds are not a particularly high-fat food, the fat they do contain is mostly healthy and beneficial for your body.
